# League Social Network using Reddit Comments
This notebook will create a social network consisting of champions that are mentioned in the same comment trees.

## Steps
1. Clean my data (trim new line characters, apostrophes, etc.)
2. Define regular expressions to use to search for occurrences of a particular champion mention. Some champions have nicknames and abbreviations that I want to account for.
3. For each comment tree, I'll create a list of champions that are mentioned within it.
4. Once I have this set of lists, I can create a count of occurrences of each unique combination and use that to create an adjacency matrix.
5. Finally, I can create my network from my adjacency matrix.

## Regular Expression (Cleaning)
First I'll clean up my data so I'm just working with words, separated by spaces. Note that this cleaning is really only meant to help with champion names (some other words will get distorted in process but I'm not too worried). 

In [97]:
import re
import json

main_dict = json.load(open('thread_comments.txt'))

In [117]:
for k,v in main_dict.items():
    for i, tree in enumerate(v):
        for j, comment in enumerate(tree): 
            comment = re.sub('(\'s)','',comment)
            comment = re.sub('[\n!?\"\'.,\\-/*\(\)]',' ',comment)
            main_dict[k][i][j] = comment